In [1]:
import os
import re
import sys

import pandas as pd

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '../ct')

import load
from preprocess import preprocess
from preprocess import Tokenizer

Using TensorFlow backend.


In [ ]:
display(f'current dir: {os.getcwd()}')

In [4]:
df = pd.read_pickle('../data/processed/treebank.pickle')

In [14]:
if not os.path.exists('../data/processed/treebank'):
    os.mkdir('../data/processed/treebank')

for i, text in enumerate(df.text):
    with open(f'../data/processed/treebank/{i + 1}', 'w') as file:
        file.write(text)

In [5]:
text = '\n\n\n\n\n\n\n\n\n\n'.join(df.text)

In [10]:
with open('../data/processed/treebank.txt', 'w', ) as file:
    file.write(text)

True

In [10]:
tokenizer = Tokenizer(input_paths=['../data/input/treebank/raw_txt/treebank-utf8.txt'],
                      tokens_output_dir='../data/processed/treebank/',
                      tokenizer_output_path='../data/tokenizer/treebank',
                      lowercase=True,
                      vocab_size=5000)

In [11]:
tokenizer


Tokenizer(vocabulary_size=5000, model=ByteLevelBPE, add_prefix_space=False, lowercase=True, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None)